In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# HispanIA DL: 1 (fastai v3)

## Competición: 'Dogs vs Cats'

Vamos a crear una solución para la competicion "Dogs vs Cats" de Kaggle. Hay 25000 imágenes de perros y gatos, de las cuales 12500 son para el validation set. 

En el momento en que la competición fue lanzada (2013), el estado del arte estaba en el 80% de precisión. Por lo que si batimos esta marca, estaríamos construyendo un modelo top del año 2013.


In [ ]:
from fastai.vision import *
from fastai.datasets import *
from fastai.metrics import *

import numpy as np
import pandas as pd
import os
import re

import matplotlib.pyplot as plt

In [ ]:
PATH = "../input/"
path_img = f'{PATH}train/'

In [ ]:
os.listdir(PATH)

## Echamos un vistazo a las imagenes

In [ ]:
# os.listdir(path_img)

In [ ]:
fnames = get_image_files(path_img)
fnames[:5]

In [ ]:
img = plt.imread(f'{fnames[-1]}')
plt.imshow(img);

Las imagenes tienen esta pinta:

In [ ]:
img.shape

In [ ]:
img[:4,:4]

## Organizando nuestros datos (imagenes y labels) mediante ImageDataBunch

In [ ]:
np.random.seed(33)
pattern = re.compile(r'/([^/]+)\.\d+.jpg$')

In [ ]:
data = ImageDataBunch.from_name_re(
    path_img, fnames, pattern, ds_tfms=get_transforms(), size=150, bs=32
                                  ).normalize(imagenet_stats)

In [ ]:
data.show_batch(rows=3, figsize=(7,6))

In [ ]:
print(data.classes)
len(data.classes),data.c

## El modelo: ResNet-34 pre-entrenado

Utilizaremos una CNN (convolutional neural network) pre-entrenada, una red neuronal **creada** y **entrenada** por otra persona que resolvía un problema de una naturaleza parecida. 

ResNet esta basado en [las capas residuales](https://github.com/KaimingHe/deep-residual-networks). Estas capas residuales son como las capas convecionales pero con la peculiaridad de que el input en crudo pasa a ser parte del output.

### Arquitectura de la ResNet-34 ### 
[Visualizacion del modelo](http://ethereon.github.io/netscope/#/gist/db945b393d40bfa26006)
![](https://cdn-images-1.medium.com/max/800/1*4tlPOipWjcwIoNUlQ6IWFQ.png)

Con tan solo **1 linea** tenemos nuestro modelo pre-entrenado:

Nota: La siguiente línea descarga de internet el modelo ResNet-34 (arquitectura y pesos) y los guarda en la variables 'learn'. Necesitamos el parámetro *path* para indicarle en qué carpeta guardar dicho modelo. 

In [ ]:
learn = create_cnn(data, models.resnet34, metrics=accuracy, path='./')

In [ ]:
learn.fit_one_cycle(1) # Aqui falta LR

In [ ]:
learn.save('stage-1')

## **98% de precisión** con unas pocas lineas, increible ##

**Con este resultado, habríamos ganado la competición de Kaggle del 2013** con unos pocos segundos y 3 líneas de código:

![](https://i.gyazo.com/b3cc85c6f5cfcd2c096fa884e6ba60ab.png)

El estado del arte previo a esta competición estaba en una precisión  del 80%. Gracias a esta competición el SOTA dió un gran salto al 98.9% de precisión. Ahora, 4 años más tarde y gracias a fastai, podemos acercarnos muchísimo e incluso batir dicho resultado en apenas unos minutos y con muy pocas lineas de código.


## Ejemplos de clasificadores de imagenes ##

* AlphaGo.

* Splunk.com: detectando transacciones fraudulentas a partir del movimiento del ratón

* Google deepmind redució  en un 40% la factura de la luz de los centros de datos de Google. [Enlace](https://deepmind.com/blog/deepmind-ai-reduces-google-data-centre-cooling-bill-40/)

* Diagnosticando cancer de pulmón
![](https://cdn-images-1.medium.com/max/800/1*_E0tiKelpZ3_7u0rOo6T5A.png)

* Otro ejemplo:

![](https://cdn-images-1.medium.com/max/800/1*BFG_B7UpS3AvJxE6lH0lug.gif)

## Deep learning: la solución 'definitiva' ##

**Una combinación de capa lineal seguida de una función no lineal a nivel de elementos nos permite crear formas arbitrariamente complejas; esta es la esencia del teorema de aproximación universal.**

![](https://cdn-images-1.medium.com/max/800/1*R4qix1l4TjKOrLkrA4t6EA.png)

## La librería fast.ai ##

Jeremy ha construido una librería potentísima en la que, con tan solo 3 líneas de código, podemos construir un clasificador de imágenes de calidad *world-class*, consiguiendo resultados top con muy pocas líneas de código.

La librería fast.ai auna todas las mejores prácticas de deep learning que van saliendo a la luz. Cada vez que sale algún nuevo paper con alguna técnica prometedora, Jeremy lo implementa y lo prueba. Si ve que funciona correctamente, lo adapta ala librería para que la podamos usar de manera super sencilla, automatizando la mayoría de cosas y encargándose de toda la parte engorrosa del deep learning.

Fast.ai está escrita sobre la librería PyTorch. La mayoría de gente la única librería que conoce es TensorFlow. Pero Jeremy dice que, hoy en día, la mayoría de investigadores que él conoce utilizan PyTorch.

Fast.ai junta todas las mejores prácticas del deep learning y las pone a disposición de todo el mundo de manera gratuita.



# Analizando e intepretando los resultados. *ClassificationInterpretation*

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

losses,idxs = interp.top_losses()

len(data.valid_ds)==len(losses)==len(idxs)

In [ ]:
interp.plot_top_losses(9, figsize=(15,11))

In [ ]:
interp.plot_confusion_matrix(figsize=(6,6), dpi=80)

In [ ]:
interp.most_confused(min_val=2)

## Unfreezing, fine-tuning y learning-rates

### Learning rate ## 
Jeremy ha implementado una técnica para encontrar el learning rate más adecuado para cada problema. Es de un paper que encontró, que no mucha gente conoce y que es muy util a la hora de afinar al máximo nuestro modelo.l

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

### Unfreeze y fine-tuning ###

Cuando hemos cargado el modelo **resnet-34** implicitamente fastai ha *congelado*  todos los layers de la red neuronal **excepto la última**. Es decir, fastai no permite que los pesos de las capas congeladas se reajusten y sólo ha dejado que se entrene la última capa del modelo.

Es improbable que las primeras capas de la ResNet-34  necesiten ser reentrenadas, ya que estas capas detectan las formas más básicas de una imagen:
* La primera capa detecta bordes
* La segunda capa reconoce curvas y esquinas.

Por lo tanto, no necesitan ser modificadas.

<img src="https://image.slidesharecdn.com/practicaldeeplearning-160329181459/95/practical-deep-learning-16-638.jpg" width="500">

Sin embargo, no ocurre lo mismo con las últimas capas, las cuales es más probable que necesiten reentrenarse

**Unfreezing**: cuando descongelamos todas las capas

In [ ]:
learn.unfreeze()

In [ ]:
learn.fit_one_cycle(2, max_lr=slice(1e-5,1e-3))

¡Ha mejorado bastante!

## Visualizando una CNN ##

![](https://cdn-images-1.medium.com/max/800/1*RPakI9UqMTYmGIm4ELhh6w.png)

[¿Cómo funciona una CNN? KERNELS](http://setosa.io/ev/image-kernels/)

## Probemos con ResNet-50

In [ ]:
data = ImageDataBunch.from_name_re(path_img, fnames, pattern, ds_tfms=get_transforms(),
                                   size=150, bs=16).normalize(imagenet_stats)

In [ ]:
learn = create_cnn(data, models.resnet50, metrics=accuracy, path='./')

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(4)

In [ ]:
learn.save('stage-1-50')

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(3, max_lr=slice(1e-5,1e-2))

## Resumen: los pasos para construir un clasificador top mundial

1. Usamos `lr_find()` para encontrar el learning rate más alto en el que el modelo está claramente mejorando.
1. Entrenar la última capa 1 o 2 epochs.
1. Learner.Unfreeze
1. Entrenar toda la red neuronal con cycle_mult=2 hasta que tengamos over-fitting

## ¿Qué es una red neuronal? ##

Lo único que tenemos que saber por ahora es que una red neuronal es una función que puede resolver cualquier problema con una precisión proporcional al número de parámetros que tenga dicha red (Teorema de Aproximación Universal)

Una red neuronal consiste en una cantidad determinada de capas de funciones lineales simples entremezcladas por otras capas de funciones no-lineales simples.

![](https://cdn-images-1.medium.com/max/800/1*0YOpyzGWkrS4VW3ntJRQ5Q.png)